In [1]:
# import libraries and navigate to directory
%cd '/home/uzumochi/eigenjuno/data/raw'
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from PIL import Image
import json, cv2, warnings, os, threading, time
from util import *
from framelets import *
from jitter_correction import *
from warnings import filterwarnings
filterwarnings('ignore')

/home/uzumochi/eigenjuno/data/raw
CSPICE_N0066


In [2]:
# pass 'names' of images to stitch here
# naming convention - 'raw_<name>.png' and 'meta_<name>.json'
thread_1 = 'test1'
thread_2 = 'test2'
thread_3 = ''
# number of threads adjustable! this is just about as much as my pc can handle

In [3]:
# stitch function for thread, taken largely from the reddit post linked in repo desc
# one change i would like to make is centering the stitching around the actual planet
# (in some outputs most of the stitched image will be just black space if image not large enough)
def stitch(name):
    image = 'raw_' + str(name) + '.png'
    im_info = 'meta_' + str(name) + '.json'
    with open(im_info, 'rb') as json_file:
        im_info_dir = json.load(json_file)

    img = Image.open(image)
    im_ar = np.array(img)
    s1, s2 = im_ar.shape

    start_time = im_info_dir["START_TIME"]
    frame_delay = float(im_info_dir["INTERFRAME_DELAY"].split()[0])+0.001

    start_correction, frame_delay = correct_image_start_time_and_frame_delay(im_ar, start_time, frame_delay)

    framelets = generate_framelets(im_ar, start_time, start_correction, frame_delay)
    # removed color correction - im_ar.astype(np.int) ** 2

    cam_pos, cam_orient = get_junocam_jupiter_rel_pos_orient(start_time, start_correction + 17 * frame_delay)

    y, x = np.mgrid[-1024:1024,-1024:1024]
    x += 300
    y += 50
    rays = np.concatenate([x[...,None], y[...,None], np.ones(((2048),(2048),1))*fl[0]], axis=-1)
    rays = rays.dot(cam_orient)

    surface_raster, _ = project_onto_jupiter_surf(cam_pos, rays)

    colors = np.zeros(((2048),(2048),3))
    color_counts = np.zeros(((2048),(2048),3))

    for k,framelet in enumerate(framelets):
        print(str(name) + ': processing framelet {} of {}..'.format(k+1, len(framelets)))
        col = framelet.color
        brightnesses, valid_map = framelet.get_pixel_val_at_surf_point(surface_raster)
        colors[...,2-col] += brightnesses
        color_counts[...,2-col] += valid_map

    colors /= np.maximum(color_counts, 1)
    colors *= 255 / np.max(colors)

    colors = colors.astype(np.uint8)

    new_img = Image.fromarray(colors)
    new_img.save("out_" + str(name) + ".png")

In [ ]:
#begin multithreaded execution
if thread_1:
    t1 = threading.Thread(target=stitch, args=(thread_1,))
    t1.start()
if thread_2:
    t2 = threading.Thread(target=stitch, args=(thread_2,))
    t2.start()
if thread_3:
    t3 = threading.Thread(target=stitch, args=(thread_3,))
    t3.start()
    
if t1:
    t1.join()
if t2:
    t2.join()
if t3: 
    t3.join()

print("done! :D")

test1: processing framelet 1 of 42..
test2: processing framelet 1 of 72..
test1: processing framelet 2 of 42..
test2: processing framelet 2 of 72..
test1: processing framelet 3 of 42..
test1: processing framelet 4 of 42..
test2: processing framelet 3 of 72..
test1: processing framelet 5 of 42..
test2: processing framelet 4 of 72..
test1: processing framelet 6 of 42..
test2: processing framelet 5 of 72..
test1: processing framelet 7 of 42..
test1: processing framelet 8 of 42..
test2: processing framelet 6 of 72..
test1: processing framelet 9 of 42..
test2: processing framelet 7 of 72..
test1: processing framelet 10 of 42..
test2: processing framelet 8 of 72..
test1: processing framelet 11 of 42..
test1: processing framelet 12 of 42..
test2: processing framelet 9 of 72..
